### The first qubit

In [ ]:
from qiskit import QuantumCircuit

# Create a quantum circuit with one qubit
qc = QuantumCircuit(1)

# Define initial_state as |1>
initial_state = [0,1]

# Apply initialization operation to the qubit at position 0
print(qc.initialize(initial_state, 0) )

dontshow=True

# <qiskit.circuit.instructionset.InstructionSet object at 0x7f59ea7f67c0>

### Prepare the simulation backend

In [ ]:
from qiskit import execute, Aer

# Tell Qiskit how to simulate our circuit
backend = Aer.get_backend('statevector_simulator')

# Do the simulation, returning the result
result = execute(qc,backend).result()

### The measured qubit

In [ ]:
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

# get the probability distribution
counts = result.get_counts()

# Show the histogram
plot_histogram(counts)

# 这是一个完全是1的概率分布

### First attempt to superpose two states

In [ ]:
# Define state |psi>
initial_state = [1, 1]

# Redefine the quantum circuit
qc = QuantumCircuit(1)

# Initialise the 0th qubit in the state `initial_state`
qc.initialize(initial_state, 0)

# execute the qc
results = execute(qc,backend).result().get_counts()

# plot the results
plot_histogram(results)
# CAPTION First attempt to superpose two states

# QiskitError: 'Sum of amplitudes-squared does not equal one.'
# 这种操作会导致失败：这是因为可能性的振幅（Amplitudes）是有一个范围的，你不能超出范围因此，下面的步骤是正规化的

### Weighted initial state

In [ ]:
from math import sqrt

# Define state |psi>
initial_state = [1/sqrt(2), 1/sqrt(2)]

# Redefine the quantum circuit
qc = QuantumCircuit(1)

# Initialise the 0th qubit in the state `initial_state`
qc.initialize(initial_state, 0)

# execute the qc
results = execute(qc,backend).result().get_counts()

# plot the results
plot_histogram(results)

### The qubit with a probability of 0.25 to result in 0

因为总体的概率分布，是1，这里初始化中，可以认为的随机给每个结果，加上一个权重

In [ ]:
from qiskit import QuantumCircuit, execute, Aer
from qiskit.visualization import plot_histogram
from math import sqrt

qc = QuantumCircuit(1)
initial_state = [1/2, sqrt(3)/2] # Here, we insert the state
qc.initialize(initial_state, 0)
backend = Aer.get_backend('statevector_simulator')
result = execute(qc,backend).result()
counts = result.get_counts()
plot_histogram(counts)

### Bypassing the Normalization
### Using theta to specify the quantum state vector

In [ ]:
from math import pi, cos, sin
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import plot_histogram

def get_state (theta):
    """returns a valid state vector"""
    return [cos(theta/2), sin(theta/2)]

# play with the values for theta to get a feeling
theta = -pi/2 # affects the probabilities
# 这个部分如果设置的夹角很小，比如除以5，那么一方（0）就会有更高更高的概率


# create, initialize, and execute the quantum circuit
qc = QuantumCircuit(1)
qc.initialize(get_state(theta), 0)
backend = Aer.get_backend('statevector_simulator')
result = execute(qc,backend).result()
counts = result.get_counts()

# Show the histogram
plot_histogram(counts)

为什么角度会影响量子态的概率？
简而言之，角度决定了量子态的叠加比例。

想象一下，量子比特就像一个旋转的硬币。当硬币处于竖直状态时，我们说它处于“0”态；当硬币水平放置时，我们说它处于“1”态。但是，硬币也可以处于任意倾斜角度，这表示量子比特处于“0”态和“1”态的叠加态。

角度代表什么？

角度的大小决定了量子比特处于“0”态和“1”态的概率。
角度越大，量子比特处于“1”态的概率越大；反之，角度越小，量子比特处于“0”态的概率越大。
数学表示

在代码中，cos(theta/2) 和 sin(theta/2) 分别代表量子比特处于“0”态和“1”态的概率幅。
概率幅的平方就是概率。所以，测量到“0”态的概率是 cos^2(theta/2)，测量到“1”态的概率是 sin^2(theta/2)。
为什么是 theta/2？

这是量子力学中的一个约定，是为了保证概率的归一化。通过这种方式，无论 theta 取什么值，cos^2(theta/2) + sin^2(theta/2) 始终等于1，即测量到“0”态或“1”态的概率之和为1。

### Circuit with measurement

测量是量子计算中获取信息的唯一方法，但它会影响量子位的状态，改变电路的计算特性。因此，量子位的测量需要在适当的时机进行，一般是在电路执行完所有计算操作之后。


In [ ]:
qc = QuantumCircuit(1)
qc.initialize(initial_state, 0)

# observe the qubit
qc.measure_all()

# Do the simulation, returning the result
result = execute(qc,backend).result()
counts = result.get_counts()
plot_histogram(counts)

# 会得到一个100%确定的结果

### Parameterized Quantum Circuit

参数化量子电路，二元分类器

在经典的机器学习中，我们通过调整参数的数值（比如权重、偏置等）来训练模型。而在量子计算中，调节**量子门**的方式其实也是类似的，只不过在量子世界里我们调的是**量子门的角度或者旋转的方向**。这些量子门就像经典机器学习中的参数，只不过它们处理的是量子态，而不是普通的数据。

**量子门的本质？**

量子门是用来对量子比特（qubit）进行操作的工具，类似于经典机器学习中的激活函数或矩阵操作，它们负责改变输入状态，生成新的输出状态。

不同于经典计算中的逻辑门（比如 AND、OR、NOT 等），量子门是 可逆 的，而且可以将量子比特从一个状态“旋转”到另一个状态。这种“旋转”是因为量子比特可以在 Bloch 球 上表示。简单来说，Bloch 球是一个用于表示量子比特状态的三维空间，量子比特的状态可以用一个向量表示，这个向量可以在球面上转动。

**为什么是角度？**

量子比特不仅仅处于经典的“0”或“1”的状态，它可以在两个状态之间的任何叠加态。通过量子门，我们可以让量子比特从一种状态“旋转”到另一种状态，这个旋转可以用一个角度来表示。

所以当我们说“调节量子门的角度”时，实际上是说我们在量子比特状态空间中，调整这个向量在 Bloch 球上的旋转角度。这个角度调节就比如对x，y，z轴进行旋转。

**量子门和经典模型参数的对比：**

1. 经典参数 vs. 量子门的旋转角度：

- 经典机器学习参数：比如权重 w 是一个数值，代表数据在某个方向上的影响力。
- 量子门角度：旋转角度 θ 也是一个数值，但它代表的是量子比特在 Bloch 球上旋转的角度，决定量子比特的状态变化。

2. 操作空间的差异：

- 经典机器学习：参数 w 控制的是数据在一个固定维度上的变化，比如在二维平面上我们调整斜率。
- 量子门：量子比特的状态在一个三维空间中变化（Bloch 球），量子门调整的角度实际上是在这个三维空间中“旋转”向量的位置。因此，调整的角度决定了量子比特的状态如何从 0 或 1，或者是它们的叠加态，转变为其他状态。

3. 线性变化 vs. 旋转变化：

- 经典神经网络：权重的调整会线性地影响模型输出的结果，比如让输出结果增加或减少。
- 量子电路：角度的变化则会引起量子比特在空间中发生旋转，导致量子态发生*非线性变化*。这种变化的方式更像是绕着某个轴转动，而不是简单的加减操作。

4. 训练方式的对比：

- 经典机器学习：你可以直接计算损失函数（比如误差），然后通过反向传播算法（backpropagation）来计算参数的梯度，调整模型。
- 量子机器学习：类似的，我们通过量子测量结果得到损失值，然后使用经典的优化算法（比如量子自然梯度下降）来调整量子门的角度。虽然是通过经典计算来优化，但调整的目标是这些量子门的旋转角度，而不是简单的数值加减。

In [ ]:
from qiskit import execute, Aer, QuantumCircuit
from math import sqrt
from sklearn.metrics import recall_score, precision_score, confusion_matrix

def pqc_classify(backend, passenger_state):
    """backend -- a qiskit backend to run the quantum circuit at
    passenger_state -- a valid quantum state vector"""

    # Create a quantum circuit with one qubit
    qc = QuantumCircuit(1)

    # Define state |Psi> and initialize the circuit
    qc.initialize(passenger_state, 0)

    # Measure the qubit
    qc.measure_all()

    # run the quantum circuit
    result=execute(qc, backend).result()

    # get the counts, these are either {'0': 1} or {'1': 1}
    counts=result.get_counts(qc)

    # get the bit 0 or 1
    return int(list(map(lambda item: item[0], counts.items()))[0])

In [ ]:
# Load the data
import numpy as np

with open('train.npy', 'rb') as f:
    train_input = np.load(f)
    train_labels = np.load(f)

with open('test.npy', 'rb') as f:
    test_input = np.load(f)
    test_labels = np.load(f)

### The scores of the random quantum classifier

In [ ]:
# REDEFINE OR IMPORT THE FUNCTIONS OF CHAPTER 2
def run(f_classify, x):
    return list(map(f_classify, x))

def specificity(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[0][1]) if (matrix[0][0]+matrix[0][1] > 0) else 0

def npv(matrix):
    return matrix[0][0]/(matrix[0][0]+matrix[1][0]) if (matrix[0][0]+matrix[1][0] > 0) else 0

def classifier_report(name, run, classify, input, labels):
    cr_predictions = run(classify, input)
    cr_cm = confusion_matrix(labels, cr_predictions)

    cr_precision = precision_score(labels, cr_predictions)
    cr_recall = recall_score(labels, cr_predictions)
    cr_specificity = specificity(cr_cm)
    cr_npv = npv(cr_cm)
    cr_level = 0.25*(cr_precision + cr_recall + cr_specificity + cr_npv)

    print('The precision score of the {} classifier is {:.2f}'
        .format(name, cr_precision))
    print('The recall score of the {} classifier is {:.2f}'
        .format(name, cr_recall))
    print('The specificity score of the {} classifier is {:.2f}'
        .format(name, cr_specificity))
    print('The npv score of the {} classifier is {:.2f}'
        .format(name, cr_npv))
    print('The information level is: {:.2f}'
        .format(cr_level))
#CAPTION A reusable function to unmask the hypocrite classifier

# The scores of the random quantum classifier
# Tell Qiskit how to simulate our circuit
backend = Aer.get_backend('statevector_simulator')

# Specify the quantum state that results in either 0 or 1
initial_state = [1/sqrt(2), 1/sqrt(2)]

classifier_report("Random PQC",
    run,
    lambda passenger: pqc_classify(backend, initial_state),
    train_input,
    train_labels)